# Загрузка данных из группы Telegram

Cмотрите шаг за шагом ниже, для получения **api_id и api_hash**

1. Подпишитесь на Telegram с помощью любого приложения.
2. Войдите в свой Telegram core:  *https://my.telegram.org*
3. Перейдите в раздел "инструменты разработки API" и заполните форму.
4. Вы получите основные адреса, а также параметры api_id и api_hash, необходимые для авторизации пользователя.

На данный момент к каждому номеру может быть подключен только один api_id.

### Ограничение по количеству строк выгрузки

In [2]:
import ipywidgets as widgets
from IPython.display import display

wlimit = widgets.IntSlider(min=1, max=1500, step=1, value=1000)
display(wlimit)
widgets.Text(value='Установите кол-во строк выгрузки', disabled=True)

IntSlider(value=1000, max=1500, min=1)

Text(value='Установите кол-во строк выгрузки', disabled=True)

### Ограничени по количество дней выгрузки

In [3]:
import ipywidgets as widgets
from IPython.display import display

wdate = widgets.IntSlider(min=1, max=30, step=1, value=1)
display(wdate)
widgets.Text(value='Установите кол-во дней выгрузки', disabled=True)

IntSlider(value=1, max=30, min=1)

Text(value='Установите кол-во дней выгрузки', disabled=True)

### Скрипт для выгрузки данных из группы
>
> Пройдите авторизацию, или двухфакторную авторизацию, если она установленна вами..

In [4]:
import os
import asyncio
from telethon import TelegramClient

from pytz import timezone
from datetime import datetime, timedelta

from tqdm import tqdm
import ipywidgets as widgets
import ipysheet

api_id = input('Скопируйте api_id: ')
api_hash = input('Скопируйте api_hash: ')

client = TelegramClient('session_name', api_id, api_hash)


ct = {
    1: 'Магазин Киото',
    2: 'Магазин Крещатик',
    3: 'Магазин Алладин',
    4: 'Магазин Марк Молл',
    5: 'Магазин Житомирская',
    6: 'Точка Гостомель',
    7: 'Точка ХарьОК',
    8: 'Точка Оазис',
    9: 'Магазин КПИ',
    10: 'Точка Малиновского',
    11: 'Точка Вишневое',
    12: 'Склад Бровары',
    13: 'Магазин Петровка',
    14: 'Склад БР22',
    15: 'Точка Буча',
    16: 'Склады',
    17: 'Точка Сталинграда',
    18: 'Точка Григоренка',
    19: 'Точка Хоткевича',
    20: 'Точка Лесі Українки',
    21: 'Точка Лятошинського',
    22: 'Точка Межигірська',
    23: 'Точка Орнамент',
    24: 'Точка Повітрофлотський',
    25: 'Точка Оболонський',
    26: 'Точка Європарк',
    27: 'Точка Боярка',
    28: 'Магазин ХШ',
    29: 'Магазин Аркадія',
    30: 'Точка Васильківська',
    31: 'Точка Ірпінь',
    32: 'Магазин Драйзера',
    33: 'Точка Бориспіль',
    34: 'Склад Мартусовка',
    35: 'Склад Подол',
    36: 'Точка Магелан',
    37: 'Точка Вишгород',
    38: 'Точка Градинська',
    39: 'Точка Липкiвського',
    40: 'Точка Москаленка',
    41: 'Точка Крюківщина',
    42: 'Точка Златоустівська',
    43: 'Точка Льва Толстого',
    44: 'Точка Дружби народів',
    45: 'Точка Кольцова',
    46: 'Точка Комп Мейтуса',
    47: 'Точка Победы 108',
    48: 'Точка Соборності',
    49: 'Точка Антоновіча',
    50: 'Точка Наб.Русановская, 4'
}


substr = {'КИО': ct[1], 'Кіот': ct[1], 'К іотто': ct[1], 'Кирто': ct[1], 'К иото': ct[1], 'Кiото': ct[1],
          'КРЕЩ': ct[2], 'ХРЕЩ': ct[2], 'Крешатик': ct[2], 'Хреш': ct[2], 'Хркщ': ct[2],
          'Киещ': ct[2], 'Коещатик': ct[2], 'щатик': ct[2],
          'АЛАД': ct[3], 'АЛЛАД': ct[3], 'Ададин': ct[3], 'Aладін': ct[3], 'Ададдін': ct[4], 'Ададін': ct[4],
          'ладін': ct[3], 'Аллдін': ct[3],
          'МАРК': ct[4], 'Mark Moll': ct[4], 'Мартмол': ct[4], 'Mark Mall': ct[4], 'Мврк Мол': ct[4],
          'мапк молл': ct[4], 'Мар мол': ct[4], 'Морк мол': ct[4],
          'ЖИТО': ct[5], 'ЖЫТОМ': ct[5], 'Жтомир': ct[5], 'Житююж': ct[5], 'омирская': ct[5], 'Житмирская': ct[5],
          'Житмирська': ct[5],
          'ГОСТ': ct[6], 'Гастомель': ct[6], 'Госномель': ct[6],
          'ХАРЬ': ct[7], 'Харёк': ct[7], 'ХАРЕК': ct[7], 'ХОР': ct[7],
          'ОАЗ': ct[8],
          'КПИ': ct[9], 'Кпі': ct[9], 'Поли': ct[9], 'Кп и': ct[9], 'К п и': ct[9], 'Kпи': ct[9], 'КПI': ct[9],
          'КІП': ct[9], 'КРІ': ct[9],
          'МАЛІН': ct[10], 'Малин': ct[10], 'Молинов': ct[10], 'еновського': ct[10], 'Малено': ct[10],
          'ВЕШН': ct[11], 'Вишнё': ct[11], 'Вишневое': ct[11], 'Вишн': ct[11], 'Вишр': ct[11],
          'БРОВАРЫ': ct[12], 'БТ СКЛАД': ct[12], 'Бровари': ct[12], 'Бр склад': ct[12],
          'ПЕТР': ct[13], 'Петровка': ct[13], 'Бандери': ct[13],
          'Бр 22': ct[14], 'Бр22': ct[14], 'Бр Ск 22': ct[14], 'Бо 22': ct[14],
          'предпринимат': ct[14], 'Бр.Склад': ct[14], 'Ск. Бр 2': ct[14],
          'Буча': ct[15],
          'VMB': ct[16],
          'Сталінг': ct[17], 'Сталин': ct[17], 'Стале': ct[17], 'инграда': ct[17], 'Стіланграда': ct[17],
          'Сиалінграда': ct[17], 'Сталінрада': ct[17],
          'Григор': ct[18], 'Грегор': ct[18], 'Грігоренко': ct[18], 'Григоге': ct[18],
          'Хоткевича': ct[19], 'Гната Хоткевича': ct[19], 'Хотке': ct[19], 'Хотуевича': ct[19],
          'Леси': ct[20], 'Л України': ct[20], 'українки': ct[20], 'Украинки': ct[20], 'Лесі': ct[20],
          'Лят': ct[21], 'Лятуш': ct[21], 'Лчтушинского': ct[21],
          'Межигорская': ct[22], 'Межиг': ct[22], 'Межы': ct[22], 'Меже': ct[22], 'гірська': ct[22], 'Межг': ct[22],
          'Орнамент': ct[23],
          'Воздух': ct[24], 'Повітро': ct[24], 'Ваозд': ct[24], 'Повітр': ct[24], 'Повiтрофлотский': ct[24],
          'трофлотський': ct[24], 'Повiтрофлотскiй': ct[24], 'Повiтроф': ct[24], 'Повiрофлотский': ct[24],
          'олонский': ct[25], 'Оболон': ct[25], 'Обол': ct[25],
          'Евро': ct[26], 'Європарк': ct[26], 'Соф': ct[26], 'Борщ': ct[26], 'Боршаговка': ct[26], 'Євро парк': ct[26],
          'Єаропарк': ct[26],
          'Бояр': ct[27],
          'Хш': ct[28], 'Х /ш': ct[28], 'Х ш': ct[28], 'Х/ш': ct[28], 'Х.ш': ct[28], 'X.Ш.': ct[28], 'Х//ш': ct[28],
          'ХЩ': ct[28],
          'Арка': ct[29],
          'Васил': ct[30], 'сильков': ct[30], 'Васиковская': ct[30],
          'Ирпень': ct[31], 'Ірп': ct[31], 'Iрпi': ct[31],
          'Драй': ct[32], 'айзера': ct[32],
          'Борис': ct[33],
          'Марту': ct[34],
          'Подол': ct[35], 'Поділ': ct[35], 'Кирилів': ct[35],
          'Магел': ct[36], 'агелан': ct[36], 'Магклан': ct[36],
          'Вишгород': ct[37], 'Вижгород': ct[37],  'Вышго': ct[37],
          'Градинська': ct[38], 'Граденська': ct[38],
          'Липкiвського': ct[39], 'Лепківська': ct[39], 'Липківського': ct[39],
          'Москал': ct[40],
          'Крюк': ct[41],
          'Злато': ct[42], 'Золото': ct[42],
          'Льва Толстого': ct[43],
          'Дружб': ct[44],
          'Кольцова': ct[45],
          'Мейтуса': ct[46],
          'Перемоги': ct[47], 'Победы 108': ct[47],
          'Соборн': ct[48],
          'Антоно': ct[49],
          'Набережная 4б': ct[50],
          }


def progress(lim):
    
    for i in tqdm(range(lim)):
        yield 0


def handler_string(text):
    tmp = []
    for b in substr.keys():
        sb = b.lower()
        string = text.lower()
        if sb in string:
            if not substr[b] in tmp:
                tmp.append(substr[b])
    return tmp


def additional_cells(jpval, date_, time_, name_, msg_, s_):
    
    ipysheet.cell(jpval, 0, jpval)
    ipysheet.cell(jpval, 1, date_)
    ipysheet.cell(jpval, 2, time_)
    ipysheet.cell(jpval, 3, name_)
    ipysheet.cell(jpval, 4, msg_)
    ipysheet.cell(jpval, 5, s_)
        

async def get_messages(days_ago=1, lim=10):

    sheet = ipysheet.sheet(rows=lim, columns=6)
    sheet.column_headers = ['index', 'Date', 'Time', 'Name', 'Point', 'S']

    prog_bar = progress(lim)
    
    days_ago = days_ago
    delta = timedelta(days=days_ago)
    fmt = "%Y-%m-%d %H:%M:%S %Z%z"
    tz_kiev = timezone('Europe/Kiev')

    tz_ = datetime.now() - delta
    tz_date = tz_.astimezone(tz=tz_kiev)
    today_str_date = datetime.strftime(tz_date, "%d.%m.%Y")
    today_date = datetime.strptime(today_str_date, "%d.%m.%Y")

    jpval = 0
    
    async with client:

        async for dialog in client.iter_dialogs():
            # print(dialog.name, 'has ID', dialog.id)
            pass

        async for message in client.iter_messages('Перемещение магазины', limit=lim):


            cur_date = message.date
            now_datetime = cur_date.astimezone(tz=tz_kiev)
            data_ukr_str = datetime.strftime(now_datetime, "%d.%m.%Y")
            gg = datetime.strptime(data_ukr_str, "%d.%m.%Y")

            if gg >= today_date:
                current_time = datetime.strftime(now_datetime, "%H:%M:%S")
                # print('*' * 20)
                msg = message.text
                try:
                    msg_list = msg.split('\n')
                    s = handler_string(msg_list[0])

                    last_name = message.sender.last_name
                    first_name = message.sender.first_name
                    name_ = ' '.join([first_name, last_name])
                    
                    additional_cells(jpval, data_ukr_str, current_time, name_, msg, s)
                    next(prog_bar)
                    jpval += 1

                except Exception as ex:
                    # print(ex, msg_list)
                    pass

            else:
                break

        print(f'Ok, End of data, limit:{lim}')
        print(f'Ok, End of data, limit:{lim}')
        print(f'Ok, End of data, limit:{lim}')
        
    return sheet

sheet = await get_messages(wdate.value, wlimit.value)  # start for Jupyter notebook

# Start without jupyter
# loop = asyncio.get_event_loop()
# loop.run_until_complete(get_messages(1))


Скопируйте api_id: 1859552
Скопируйте api_hash: 22d9476bbbcca5a726b214d085206b67


 72%|██████████████████████████████████████████████████████████▋                       | 48/67 [00:00<00:00, 48.44it/s]

Ok, End of data, limit:67
Ok, End of data, limit:67
Ok, End of data, limit:67


### Скачать данные в CSV file

In [5]:
import io
import openpyxl
from pandas import ExcelFile

import base64
from IPython.display import HTML
from IPython.display import FileLink

import warnings
warnings.filterwarnings('ignore')

def create_download_link(df, title = "Скачать данные в CSV file", filename = "data.csv"):  
    csv = df.to_csv(sep=';', encoding='utf-8-sig', index=False)
    b64 = base64.b64encode(csv.encode('utf-8-sig'))
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

new_df = ipysheet.to_dataframe(sheet)
new_df.pop('index')

create_download_link(new_df)